In [17]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from math import floor, ceil


def boximage(box_width, box_height, width = 200, height = 200, center = None):
    if center is None:
        center = ((height-1)/2, (width-1)/2)


    im = np.zeros((height, width, 1), dtype=np.uint8)

    y_start = max(center[0]-box_height/2, 0)
    y_end = min(center[0]+box_height/2, height-1)

    x_start = max(center[1]-box_width/2, 0)
    x_end = min(center[1]+box_width/2, width-1)

    # Draw the edges
    im[floor(y_start), ceil(x_start):floor(x_end+1)] = 255*(ceil(y_start)-y_start)
    im[ceil(y_end), ceil(x_start):floor(x_end+1)] = 255*(y_end - floor(y_end))
    im[ceil(y_start):floor(y_end+1), floor(x_start)] = 255*(ceil(x_start) - x_start)
    im[ceil(y_start):floor(y_end+1), ceil(x_end)] = 255*(x_end - floor(x_end))

    # Draw the four corners
    im[floor(y_start), floor(x_start)] = 255*(ceil(y_start) - y_start)*(ceil(x_start) - x_start)
    im[floor(y_start), ceil(x_end)] = 255*(ceil(y_start) - y_start)*(x_end - floor(x_end))
    im[ceil(y_end), floor(x_start)] = 255*(y_end - floor(y_end))*(ceil(x_start) - x_start)
    im[ceil(y_end), ceil(x_end)] = 255*(y_end - floor(y_end))*(x_end - floor(x_end))

    # Fill the box
    im[ceil(y_start):floor(y_end+1), ceil(x_start):floor(x_end+1)] = 255


    return im

box_widths = np.linspace(0, 100, 200)
box_heights = np.linspace(10, 90, 200)
center_x = np.linspace(0, 200, 200)
center_y = np.linspace(0, 200, 200)
video = np.array([boximage(box_width, box_height, center = (cx, cy)) for box_width, box_height, cx, cy in zip(box_widths, box_heights, center_x, center_y)])

np.save('boxvideo.npy', video)

#animate video with matplotlib.funcanimation
def animate_video(video):
    %matplotlib tk

    fig, ax = plt.subplots(1, 1, figsize=(5, 10))

    def animate(frame):
        ax.clear()
        ax.imshow(video[frame], cmap='gray')
        ax.set_title(f'Frame {frame}')

    ani = animation.FuncAnimation(fig, animate, frames=range(len(video)), interval=5, repeat=False)

    return ani

ani = animate_video(video)


In [16]:
video.shape

(200, 200, 200, 1)

In [63]:
#autoencoder
import keras
from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Dense, Flatten, Reshape

model = Sequential()
model.add(Flatten())
model.add(Dense(400))
model.add(Dense(40))
model.add(Dense(400))
model.add(Dense(200*200))
model.add(Reshape((200, 200)))
model.compile(optimizer='adam', loss='mean_squared_error')
model.build((None, 200, 200, 1))

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_10 (Flatten)        (None, 40000)             0         
                                                                 
 dense_40 (Dense)            (None, 400)               16000400  
                                                                 
 dense_41 (Dense)            (None, 40)                16040     
                                                                 
 dense_42 (Dense)            (None, 400)               16400     
                                                                 
 dense_43 (Dense)            (None, 40000)             16040000  
                                                                 
 reshape_10 (Reshape)        (None, 200, 200)          0         
                                                                 
Total params: 32,072,840
Trainable params: 32,072,840

In [64]:
#fit
model.fit(video, video, epochs=100, batch_size=32)

Epoch 1/100
16/16 [==============================] - 1s 67ms/step - loss: 2952.3547
Epoch 2/100
16/16 [==============================] - 1s 58ms/step - loss: 544.0809
Epoch 3/100
16/16 [==============================] - 1s 57ms/step - loss: 227.3054
Epoch 4/100
16/16 [==============================] - 1s 59ms/step - loss: 147.8180
Epoch 5/100
16/16 [==============================] - 1s 57ms/step - loss: 111.2778
Epoch 6/100
16/16 [==============================] - 1s 59ms/step - loss: 98.3800
Epoch 7/100
16/16 [==============================] - 1s 57ms/step - loss: 87.1685
Epoch 8/100
16/16 [==============================] - 1s 60ms/step - loss: 74.4603
Epoch 9/100
16/16 [==============================] - 1s 57ms/step - loss: 66.4866
Epoch 10/100
16/16 [==============================] - 1s 56ms/step - loss: 59.2644
Epoch 11/100
16/16 [==============================] - 1s 56ms/step - loss: 52.9074
Epoch 12/100
16/16 [==============================] - 1s 57ms/step - loss: 48.9242
Epoch 1

In [61]:
pred_video = model.predict(video)
animate_video(pred_video)

16/16 [==============================] - 0s 5ms/step


In [ ]:
pred_video.shape

In [ ]:
video.shape

# Import saved AE

In [5]:
from src.models.AE import Encoder, Decoder, AE
from src.utils.misc import load_config, animate_video
import torch

def build_model(model_type: str, CFG, device: str):
    H = CFG['data']['H']
    W = CFG['data']['W']
    latent_dim = CFG['AE']['D']
    if model_type == 'AutoEncoder':
        encoder = Encoder(H, W, latent_dim)
        decoder = Decoder(H, W, latent_dim)
        model = AE(encoder, decoder)
    else:
        raise ValueError(f"Unsupported model type: {model_type}")
    return model



CFG = load_config('configs/config.yaml')
device = ('cuda' if torch.cuda.is_available() else 'cpu')
model = build_model('AutoEncoder', {'data': {'H': 200, 'W': 200}, 'AE': {'D': 40}}, 'cpu')
model.load_state_dict(torch.load('data/model_weights/AutoEncoder_weights.pt', map_location=device))



<All keys matched successfully>

# Perform frame interpolation


In [6]:
def interpolate_linear(observed_times, new_times, states):
    new_states = np.zeros((len(new_times), *states.shape[1:]))

    for time in new_times:
        # find the two closest observed times
        idx = np.searchsorted(observed_times, time)
        if idx == 0:
            new_states[time] = states[0]
        elif idx == len(observed_times):
            new_states[time] = states[-1]
        else:
            # linearly interpolate between the two closest observed times
            alpha = (time - observed_times[idx-1]) / (observed_times[idx] - observed_times[idx-1])
            new_states[time] = alpha * states[idx] + (1 - alpha) * states[idx-1]

    return new_states


interpolated_video = interpolate_linear(np.arange(len(video)), np.arange(0, len(video), 0.1), video)
